In [1]:
import sys
sys.path.insert(0, "../src/gen")
sys.path.insert(1, "../src/rte")
from pathlib import Path
from collections import Counter

import numpy as np
from scipy.special import softmax

from util import read_data, write_jsonl
from aggregate import generate_micro_macro_df

In [2]:
# constants
LOOKUP = {
    "verifiable": {"no": "NOT VERIFIABLE", "yes": "VERIFIABLE"},
    "label": {"nei": "NOT ENOUGH INFO", "r": "REFUTES", "s": "SUPPORTS"}
}

SEED = 123456789

LABEL2ID = {"SUPPORTS": 0, "NOT ENOUGH INFO": 1, "REFUTES": 2}
ID2LABEL = {0: "SUPPORTS", 1: "NOT ENOUGH INFO", 2: "REFUTES"}

# Init

In [3]:
import evaluate
from datasets import Dataset, DatasetDict
from transformers import (
    AutoTokenizer, 
    AutoModelForSequenceClassification,
    TrainingArguments, 
    Trainer,
    DataCollatorWithPadding,
    TextClassificationPipeline,
    pipeline
)

# Huggingface Init

## Model

In [4]:
accuracy_metric = evaluate.load("accuracy")
recall_metric = evaluate.load("recall")
precision_metric = evaluate.load("precision")
f1_metric = evaluate.load("f1")

In [5]:
model_checkpoint = "bert-base-uncased"
def model_init():
    return AutoModelForSequenceClassification.from_pretrained(
        model_checkpoint, 
        num_labels=3, 
        id2label=ID2LABEL, 
        label2id=LABEL2ID
    )

model = model_init()
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, do_lower_case=True)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

def preprocess(examples):
    return tokenizer(examples["evidence"], examples["claim"], max_length=512, truncation="only_first")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    
    results = {}
    results.update(accuracy_metric.compute(predictions=predictions, references=labels))
    results.update(recall_metric.compute(predictions=predictions, references=labels, average="macro"))
    results.update(precision_metric.compute(predictions=predictions, references=labels, average="macro"))
    results.update(f1_metric.compute(predictions=predictions, references=labels, average="macro"))
    
    return results

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [6]:
dataset = ["fever", "climatefeverpure", "fever-climatefeverpure", "climatefever", "fever-climatefever"]
task = ["hp_tune", "output"]
doc_sent = ["doc", "sent"]

di = 2
ti = 0
ds = 1

model_store_path = Path("/users/k21190024/study/fact-check-transfer-learning/scratch/thesis/models").joinpath(model_checkpoint)
model_store_path.mkdir(exist_ok=True)
model_store_path = model_store_path / f"{dataset[di]}-{model_checkpoint}-{doc_sent[ds]}-{task[ti]}"

## Dataset

In [7]:
datap = Path(f"/users/k21190024/study/fact-check-transfer-learning/scratch/dumps/bert-data-{doc_sent[ds]}-evidence")

data = DatasetDict({
    "train": Dataset.from_list(read_data(datap / f"{dataset[di]}.train.n5.jsonl")),
    "validation": Dataset.from_list(read_data(datap / f"{dataset[di]}.dev.n5.jsonl")),
    "fever_test": Dataset.from_list(read_data(datap / f"{dataset[0]}.test.n5.jsonl")),
    "climatefever_test": Dataset.from_list(read_data(datap / f"{dataset[1]}.test.n5.jsonl"))
}).map(preprocess, batched=True)

Map:   0%|          | 0/256386 [00:00<?, ? examples/s]

Map:   0%|          | 0/18326 [00:00<?, ? examples/s]

Map:   0%|          | 0/17327 [00:00<?, ? examples/s]

Map:   0%|          | 0/459 [00:00<?, ? examples/s]

## Trainer

In [8]:
batch_size = 16
learning_rate = 2e-5
epoch = 4
metric_name = "f1"
warmup_ratio=0.1
save_steps=200
eval_steps=200

# Hyperparameter tuning

In [9]:
# shard the data if the dataset is large for hyperparameter tuning
shard = data["train"].num_rows > 50000
hp_tune_train = data["train"].shuffle(seed=SEED).shard(num_shards=5, index=1)

In [10]:
training_args = TrainingArguments(
    model_store_path,
    overwrite_output_dir=True,
    evaluation_strategy = "steps",
    eval_steps=eval_steps,
    save_strategy = "no",
    learning_rate=learning_rate,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=epoch,
    weight_decay=0.01,
    metric_for_best_model=metric_name,
    push_to_hub=False,
    seed=SEED,
    data_seed=SEED,
    warmup_ratio=warmup_ratio,
    report_to="tensorboard"
)

_ = model.train()
trainer = Trainer(
    model_init=model_init,
    args=training_args,
    train_dataset=hp_tune_train if shard else data["train"],
    eval_dataset=data["validation"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [11]:
def optuna_hp_space(trial):
    return {
        "learning_rate": trial.suggest_categorical("learning_rate", [5e-5, 3e-5, 2e-5]),
        "per_device_train_batch_size": trial.suggest_categorical("per_device_train_batch_size", [16, 32]),
        "num_train_epochs": trial.suggest_int("num_train_epochs", 1, 5),
    }

def compute_objective(metrics):
    return metrics["f1"]

In [12]:
best_run = trainer.hyperparameter_search(
    direction="maximize", 
    n_trials=20, 
    hp_space=optuna_hp_space
)

[I 2023-07-02 22:24:20,485] A new study created in memory with name: no-name-8cdc174e-8dd2-45b5-99f7-65e9bbd09896
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClas

Step,Training Loss,Validation Loss,Accuracy,Recall,Precision,F1
200,No log,0.709205,0.664948,0.630959,0.465465,0.520997
400,No log,0.407057,0.833909,0.823710,0.831642,0.817921
600,0.521500,0.339742,0.873279,0.866250,0.877925,0.862448
800,0.521500,0.309051,0.890386,0.883398,0.890336,0.881781
1000,0.229500,0.258453,0.911184,0.903836,0.908845,0.904794
1200,0.229500,0.281829,0.904447,0.901484,0.901296,0.898728
1400,0.229500,0.247656,0.914055,0.906301,0.912077,0.908129
1600,0.194000,0.295872,0.911008,0.905235,0.907248,0.904746
1800,0.194000,0.268496,0.914582,0.909214,0.910846,0.908957
2000,0.133700,0.295163,0.913352,0.909440,0.909210,0.907789


/scratch/users/k21190024/envs/conda/p-dis-torch/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
[I 2023-07-02 22:35:48,568] Trial 0 finished with value: 3.6577068518348232 and parameters: {'learning_rate': 2e-05, 'per_device_train_batch_size': 32, 'num_train_epochs': 2}. Best is trial 0 with value: 3.6577068518348232.
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias']
- This IS expected if you are

Step,Training Loss,Validation Loss,Accuracy,Recall,Precision,F1
200,No log,0.815616,0.615150,0.590411,0.446991,0.486206
400,No log,0.736004,0.655750,0.624416,0.464368,0.515318
600,0.676600,0.610219,0.673736,0.641676,0.744473,0.543583
800,0.676600,0.405915,0.843576,0.833540,0.831153,0.831706
1000,0.388600,0.374655,0.865839,0.858709,0.859833,0.855843
1200,0.388600,0.360239,0.883824,0.879217,0.880916,0.876090
1400,0.388600,0.290826,0.900873,0.895359,0.893537,0.894380
1600,0.250500,0.297646,0.898061,0.890760,0.896296,0.890459
1800,0.250500,0.305376,0.901400,0.895757,0.899722,0.894439
2000,0.231300,0.317847,0.900521,0.890517,0.900745,0.892190


/scratch/users/k21190024/envs/conda/p-dis-torch/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/scratch/users/k21190024/envs/conda/p-dis-torch/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
[I 2023-07-02 23:09:34,812] Trial 1 finished with value: 3.6645075218228365 and parameters: {'learning_rate': 2e-05, 'per_device_train_batch_size': 16, 'num_train_epochs': 4}. Best is trial 1 with value: 3.6645075218228365.
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceC

...

/scratch/users/k21190024/envs/conda/p-dis-torch/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
[I 2023-07-03 00:46:47,793] Trial 17 pruned. 
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification

Step,Training Loss,Validation Loss,Accuracy,Recall,Precision,F1
200,No log,0.709205,0.664948,0.630959,0.465465,0.520997
400,No log,0.407057,0.833909,0.823710,0.831642,0.817921


/scratch/users/k21190024/envs/conda/p-dis-torch/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
[I 2023-07-03 00:48:25,015] Trial 18 pruned. 
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification

Step,Training Loss,Validation Loss,Accuracy,Recall,Precision,F1
200,No log,0.756087,0.636827,0.608787,0.457892,0.502235


/scratch/users/k21190024/envs/conda/p-dis-torch/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
[I 2023-07-03 00:49:01,721] Trial 19 pruned. 


In [13]:
best_run

BestRun(run_id='1', objective=3.6645075218228365, hyperparameters={'learning_rate': 2e-05, 'per_device_train_batch_size': 16, 'num_train_epochs': 4}, run_summary=None)

## Train with best hyperparameters

In [9]:
training_args = TrainingArguments(
    model_store_path,
    overwrite_output_dir=True,
    evaluation_strategy = "steps",
    eval_steps=1000,
    save_strategy = "steps",
    save_steps=1000,
    save_total_limit=5,
    learning_rate=learning_rate,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=epoch,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model=metric_name,
    push_to_hub=False,
    seed=SEED,
    data_seed=SEED,
    warmup_ratio=warmup_ratio,
    report_to="tensorboard"
)

trainer = Trainer(
    model_init=model_init,
    args=training_args,
    train_dataset=data["train"],
    eval_dataset=data["validation"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

for n, v in best_run.hyperparameters.items():
    setattr(trainer.args, n, v)

trainer.train()

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

Step,Training Loss,Validation Loss,Accuracy,Recall,Precision,F1
1000,0.583100,0.669944,0.669650,0.638418,0.476781,0.532022
2000,0.284900,0.356390,0.875641,0.868001,0.878602,0.866587
3000,0.199100,0.266495,0.914930,0.910840,0.914062,0.910681
4000,0.190400,0.305778,0.908764,0.904308,0.911469,0.904030
5000,0.180200,0.291843,0.913511,0.909202,0.916994,0.908917
6000,0.185800,0.206423,0.935720,0.932371,0.934348,0.932861
7000,0.159500,0.265357,0.926007,0.919992,0.928611,0.922205
8000,0.163200,0.243510,0.933428,0.931470,0.931583,0.930966
9000,0.163400,0.278273,0.924151,0.920673,0.922975,0.920614
10000,0.153800,0.250510,0.934792,0.931761,0.933082,0.932001


/scratch/users/k21190024/envs/conda/p-dis-torch/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



TrainOutput(global_step=64100, training_loss=0.09774291333467838, metrics={'train_runtime': 7322.2335, 'train_samples_per_second': 140.059, 'train_steps_per_second': 8.754, 'total_flos': 5.871476434148186e+16, 'train_loss': 0.09774291333467838, 'epoch': 4.0})

In [10]:
trainer.save_model(model_store_path.parent / (model_store_path.stem + ".out"))

# Evaluate

In [11]:
import pandas as pd
from sklearn.metrics import classification_report

## Test on validation data

In [12]:
preds = trainer.predict(data["validation"])
micro_val, macro_val = generate_micro_macro_df(data["validation"], preds)

In [13]:
print(classification_report(y_true=micro_val["actual"], y_pred=micro_val["predicted"]))

                 precision    recall  f1-score   support

NOT ENOUGH INFO       0.96      0.98      0.97      7141
        REFUTES       0.95      0.91      0.93      5479
       SUPPORTS       0.92      0.95      0.93      5706

       accuracy                           0.95     18326
      macro avg       0.95      0.94      0.94     18326
   weighted avg       0.95      0.95      0.95     18326



## Test on test data

### FEVER

In [16]:
preds = trainer.predict(data["fever_test"])
micro_val, macro_val = generate_micro_macro_df(data["fever_test"], preds)

In [17]:
print(classification_report(y_true=micro_val["actual"], y_pred=micro_val["predicted"]))

                 precision    recall  f1-score   support

NOT ENOUGH INFO       0.98      0.99      0.98      6666
        REFUTES       0.96      0.88      0.92      5272
       SUPPORTS       0.90      0.96      0.93      5389

       accuracy                           0.95     17327
      macro avg       0.95      0.95      0.95     17327
   weighted avg       0.95      0.95      0.95     17327



### Climate-FEVER

In [20]:
preds = trainer.predict(data["climatefever_test"])
micro_val, macro_val = generate_micro_macro_df(data["climatefever_test"], preds)

In [24]:
print(classification_report(y_true=micro_val["actual"], y_pred=micro_val["predicted"]))

                 precision    recall  f1-score   support

NOT ENOUGH INFO       0.68      0.77      0.72       235
        REFUTES       0.51      0.42      0.46        48
       SUPPORTS       0.68      0.59      0.63       176

       accuracy                           0.66       459
      macro avg       0.62      0.59      0.60       459
   weighted avg       0.66      0.66      0.66       459

